In [1]:
import pandas as pd
import random

In [243]:
class Card:
    def __init__(self,age,name,cost,build,product,color):
        self.age = age
        self.name = name
        self.cost = cost.split(',')
        self.build = build.split(',')
        self.product = product.split(',')
        self.color = color
        self.trade_cost = {"left":False,"right":False}
        self.is_available = False
    
    def view(self):
        print(f"Age: {self.get_age(True)}")
        print(f"Name: {self.get_name(True)}")
        print(f"Cost: {self.get_cost(True)}")
        print(f"Chain: {self.get_build(True)}")
        print(f"Product: {self.get_product(True)}")
        print(f"Color: {self.get_color(True)}")
        
    def set_cost(self, cost):
        self.cost = cost

    def get_age(self, view=False):
        return self.age
    
    def get_name(self, view=False):
        return self.name
    
    def get_cost(self, view=False):
        if(view):
            return ', '.join(self.cost)
        return self.cost
    
    def get_build(self, view=False):
        if(view):
            return ', '.join(self.build)
        return self.build
    
    def get_product(self, view=False):
        if(view):
            return ', '.join(self.product)
        return self.product
    
    def get_color(self, view=False):
        return self.color

In [244]:
class Board:
    def __init__(self,name,start,num_stages,costs,rewards,time):
        self.name = name.iloc[0]
        self.start = start
        self.num_stages = int(num_stages.iloc[0])
        self.costs = costs.to_string(index=False).split("-")
        self.rewards = self.set_rewards(rewards)
        self.time = time.to_string(index=False)
        self.stage = 0
        
    def view(self):
        print(f"{self.name}")
        print(f"Starting resource: {self.start.to_string(index=False)}")
        print(f"Time: {self.time}")
        for x in range(1,self.num_stages + 1):
            built = ""
            if self.stage + 1 >= x:
                built = " (built)"
            print(f"Stage {x}{built}: \n\tCost: {self.get_stage_cost(x - 1, view=True)}\n\tReward: {self.get_stage_reward(x - 1, view=True)}")
                
        
    def get_stage(self, view=False):
        return self.stage
    
    def get_start(self):
        return self.start.to_string(index=False)
    
    def get_stage_cost(self,stage, view=False):
        if(view):
            return ', '.join(self.costs[stage].split(","))
        return self.costs[stage].split(",")
    
    def get_stage_reward(self,stage, view=False):
        if(view):
            return ', '.join(self.rewards[stage].split(","))
        return self.rewards[stage].split(",")
    
    def get_name(self):
        return self.name
    
    def next_stage(self):
        self.stage += 1
    
    def set_rewards(selfm,rewards):
        rewards = rewards.to_string(index=False).split("-")

        for i,r in enumerate(rewards):
            num_victory = 0
            num_coin = 0
            if r[:2] == "vp":
                num_victory = int(r[2])
            vp = ""
            for victory in range(0,num_victory-1):
                vp = vp + "victory,"
            if vp:
                vp = vp + "victory"
                rewards[i] = vp
            if r[:2] == "au":
                num_coin = int(r[2])
            au = ""
            for coin in range(0,num_coin-1):
                au = au + "coin,"
            if au:
                au = au + "coin"
                rewards[i] = au
        return rewards

In [ ]:
class Player:
    def __init__(self,name,board,ai=False):
        self.name=name
        self.board=board
        self.is_ai=ai
        self.hand={}
        self.available_cards = {}
        self.unavailable = {}
        self.stage = {"available":False, "trade":{}}
        self.resources={"wood":0,"stone":0,"brick":0,"ore":0,"glass":0,"silk":0,"paper":0,"coin":3, "any":0, "any_rare":0, "choice":{}}
        self.special_resources={"victory":0,"military":0,"cog":0,"compass":0,"tablet":0, "any_science":0, "tokens":{}}
        self.add_resources([board.get_start()], is_card=False)
        self.build = {"free":0, "none":0}
        self.trade_cards = {}
        self.left_neighbor = ""
        self.right_neighbor = ""
        self.is_trading_left = False
        self.is_trading_right = False
        self.left_coins = 0
        self.right_coins = 0
        
        
        
    def set_hand(self,hand):
        self.available_cards = {}
        self.trade_cards = {}
        self.unavailable = {}
        self.left_coins = 0
        self.right_coins = 0
        self.hand = hand
        
    def set_left(self,player):
        self.left_neighbor = player
        
    def set_right(self,player):
        self.right_neighbor = player
    
    def set_action(self):
        card_name = ""
        action = 0
        stage_available = "not available"
        
        if not self.available_cards:
            self.available(self.hand)
            if self.stage["available"]:
                stage_available = "available"
        print(f"trade______________________ {self.trade_cards}")
        print(f"fjlwlnkdwawdkwdkwdkwk {self.available_cards}")
        
        if not self.is_ai:
            self.view_hand()
            confirmed = False
            while not confirmed:
                print(f"""Select Action:\n1) play (card)\n2) discard (card)\n3) build wonder (card) ({stage_available})
                \n4) view hand\n5) view board\n6) view resources
                \n7) view {self.left_neighbor.get_name()}'s board (left)\n8) view {self.right_neighbor.get_name()}'s board (right)""")
                action = input()
                if not action.isnumeric() or int(action) > 8 or int(action) < 1:
                    print("please select action with 1, 2, 3, 4, 5, 6, 7 or 8")
                else:
                    action = int(action)
                    if action < 4:
                        confirmed, card_name = self.set_play_action(action)
                    elif action == 4:
                        self.view_hand()
                    elif action == 5:
                        self.board.view()
                    elif action == 6:
                        self.get_resources(view=True)
                    elif action == 7:
                        self.left_neighbor.board.view()
                        self.left_neighbor.get_resources(view=True)
                    elif action == 8:
                        self.right_neighbor.board.view()
                        self.right_neighbor.get_resources(view=True)

        else:
            if self.stage["available"] and len(self.hand) < 3:
                card_name = random.choice(list(self.hand))
                action = 3
            elif self.available_cards:
                card_name = random.choice(list(self.available_cards))
                action = 1 #play
            elif self.trade_cards:
                card_name = random.choice(list(self.trade_cards))
                action = 1 #trade
            else:
                card_name = random.choice(list(self.unavailable_cards))
                action = 2
        
        return action, card_name
        
    def set_play_action(self, action):
        action_list = {1:"play", 2:"discard", 3:"build wonder"}
        valid = False
        conf_action = ""
        card_name = ""
        while not valid:
            if action == 1 and (not self.available_cards and not self.trade_cards):
                print("There are no available cards to play")
                self.set_action()
                return
            if action == 1  or action == 2:
                card_name = input(f"what card do you want to {action_list[action]}? ")
                if card_name in self.hand:
                    if card_name in self.trade_cards and action == 1:
                        if self.confirm_trade(card_name):
                            conf_action = action_list[action] + " " + card_name
                            valid = True
                    elif card_name in self.unavailable and action == 1:
                        print(f"Not enough resources to {action_list[action]} {card_name}")
                    else:
                        conf_action = action_list[action] + " " + card_name
                        valid = True
                        
                
            elif action == 3:
                if self.stage["available"]:
                    if self.stage["trade"]:
                        self.confirm_trade("stage")
                    card_name = input(f"what card do you want to discard to build {self.board.get_stage()}? ")
                    conf_action = action_list[action] + "by discarding " + card_name
                    valid = True
                else:
                    print("Unable to build next wonder stage")
            if card_name not in self.hand:
                print("invalid card")
        
        confirmed = False
        while not confirmed:
            confirm = input(f"confirm action: {conf_action}? y/n ")
            if(confirm.lower() == "y" or confirm.lower() == "yes" or confirm == "0"):
                confirmed = True
                print("confirmed")
            elif confirm.lower() == "n" or confirm.lower() == "no" or confirm == "1":
                print("action canceled")
                return confirmed, card_name
            else:
                print("please respond with yes or no")
        
        return confirmed, card_name
            
    def play_card(self,action, card_name):
        card = self.hand[card_name]
        if action == 1:
            if card_name in self.available_cards or card_name in self.trade_cards:
                self.add_resources(card)
                if "coin" in card.get_cost(): # only time when resources are taken
                    for cost in card.get_cost():
                        if cost == "coin":
                            self.resources["coin"] -= 1
        elif action == 2:
            self.add_resources(["coin","coin","coin"], is_card=False)
        else:
            stage = self.board.get_stage()
            board_reward = self.board.get_stage_reward(stage)
            self.add_resources(board_reward, is_card=False)
            self.board.next_stage()
            self.stage["available"] = False
            self.stage["trade"] = {}
            
        del self.hand[card_name]
        return self.hand
    
    def confirm_trade(self, card_name):
        check = self.hand[card_name]
        left_neighbor = self.left_neighbor.get_name()
        right_neighbor = self.right_neighbor.get_name()
        rare_resource = ["glass","silk","paper"]
        needed_resources = {"wood":0,"stone":0,"brick":0,"ore":0,"glass":0,"silk":0,"paper":0}
        left_check = self.trade_cards[card_name]["left"].copy()
        right_check = self.trade_cards[card_name]["right"].copy()
        coin_check = self.resources["coin"]
        right_coins = 0
        left_coins = 0
        self.left_coins = 0
        self.right_coins = 0
        trade_resources = ""
        
        if card_name == "stage":
            trade_resources = self.stage["trade"]
        elif card_name in self.trade_cards:
            trade_resources = self.trade_cards[card_name]
        else:
            print("error")
            return False
            
        for r in trade_resources:
            for left_cost in left_check:
                while left_check[left_cost] > 0 and left_cost != "market" and left_cost != "post":
                    if right_check and right_check[left_cost] > 0:
                        right_check[left_cost] -= 1
                    left_check[left_cost] -= 1
                    needed_resources[left_cost] += 1
                    
            for right_cost in right_check:
                while right_check[right_cost] > 0 and right_cost != "market" and right_cost != "post":
                    right_check[right_cost] -= 1
                    needed_resources[right_cost] += 1
        
        left_check = self.trade_cards[card_name]["left"].copy()
        right_check = self.trade_cards[card_name]["right"].copy()
        trade_partners = {left_neighbor:left_check, right_neighbor:right_check} 
        trade_partner = ""
        if not self.is_ai:
            for r in needed_resources:
                while needed_resources[r] > 0:
                    print(f"You need to trade for {needed_resources[r]} more {r}")
                    picked = False
                    if left_check and right_check:
                        print(f"{left_neighbor} has {left_check[r]} {r} and {right_neighbor} has {right_check[r]} {r}.")
                    elif left_check:
                        print(f"Only {left_neighbor} has {r} ({left_check[r]}).")
                        trade_partner = left_neighbor
                        picked = True
                    else:
                        print(f"Only {right_neighbor} has {r} ({right_check[r]}).")
                        trade_partner = right_neighbor
                        picked = True
                    
                    while not picked:
                        trade_partner = input(f"From whom do you want to trade one {r} from? {left_neighbor} or {right_neighbor}? (0 to cancel trade) ")
                        if trade_partner == "0": #cancel trade
                            print("trade canceled")
                            self.set_action()
                            return False
                        elif trade_partner != left_neighbor and trade_partner != right_neighbor:
                            print(f"Can't recongnize {trade_partner}")
                        elif trade_partners[trade_partner][r] == 0:
                            print(f"{trade_partner} does not have enough {r} to trade")
                    
                    neighbor = "left"
                    if trade_partner == right_neighbor:
                        neighbor = "right"
                    trade_cost = trade_partners[trade_partner]["post"]
                    if r in rare_resource:
                        trade_cost = trade_partners[trade_partner]["market"]
                    print(f"It costs {trade_cost} coins to trade from {trade_partner}")

                    if trade_cost > coin_check:
                        print(f"Not enough coins to trade. Pick other neighbor or cancel trade to reset")
                    else:
                        confirm = input(f"Confirm? type 1 to confirm. (You have {coin_check} coins) ")
                        if confirm == "1" and trade_cost <= self.resources["coin"]:
                            picked = True
                            coin_check -= trade_cost
                            needed_resources[r] -= 1
                            if trade_partner == left_neighbor:
                                left_check[r] -= 1
                                left_coins += trade_cost
                            else:
                                right_check[r] -= 1
                                right_coins += trade_cost
                        else:
                            print(f"trade unconfirmed")

        else:
            trade_partner = self.get_trade_partner(left_check, right_check)
            check = trade_cards[card_name][trade_partner].copy()
            for r in needed_resources:    
                while needed_resources[r] > 0:
                    if check[r] == 0:
                        trade_partner = self.get_trade_partner(left_check, right_check, r, True if r in rare_resource else False)
                        check = trade_cards[card_name][trade_partner].copy()
                    else:
                        needed_resources[r] -= 1
                        trade_cost = trade_cards[card_name][trade_partner]["post"]
                        if r in rare_resource:
                            trade_cost = trade_cards[card_name][trade_partner]["market"]
                            
                        if trade_partner == "left":
                            left_check[r] -= 1
                            left_coins += trade_cost
                        else:
                            right_check[r] -= 1
                            right_coins += trade_cost
                    
        confirm_trade = False
        while not confirm_trade:
            confirm = "yes"
            if not self.is_ai:
                confirm = input(f"It will cost {left_coins + right_coins} coins to play {card_name}. 0 to cancel, anything else to confirm ")
            if confirm == "0": #cancel trade
                print("trade canceled")
                self.set_action()
                return False
            else:
                if left_coins > 0:
                    self.is_trading_left = True
                    self.left_coins = left_coins
                if right_coins > 0:
                    self.is_trading_right = True
                    self.right_coins = right_coins
                confirm_trade = True
        
        return True
    
    def get_trade_partner(self, left, right, check="", is_rare=False):
        left_none = True
        right_none = True
        
        if check:
            if left[check] > 0:
                left_none = False
            if right[check] > 0:
                right_none = False
        else:
            for r in left:
                if left[r] != 0:
                    left_none = False
        
        
        trade_partner = "left"
        if is_rare:
            if trade_cards[card_name]["left"]["market"] == trade_cards[card_name]["right"]["market"] and (not left_none and not right_none):
                trade_partner = random.choice(["left", "right"])
            elif trade_cards[card_name]["left"]["market"] > trade_cards[card_name]["right"]["market"] or left_none:
                trade_partner = "right"
        else:
            if trade_cards[card_name]["left"]["post"] == trade_cards[card_name]["right"]["post"] and (not left_none and not right_none):
                trade_partner = random.choice(["left", "right"])
            elif trade_cards[card_name]["left"]["post"] > trade_cards[card_name]["right"]["post"] or left_none:
                trade_partner = "right"
        
        return trade_partner
        
    def get_neighbor(self, neighbor = "left"):
        if neighbor == "left":
            return self.left_neighbor
        return self.right_neighbor
    
    def get_name(self):
        return self.name
    
    def get_resources(self, special=False, view=False):
    
        if view:
            print(f"{self.name}'s resources:")
            for r in self.resources:
                if not isinstance(self.resources[r],int) and len(self.resources[r]) != 0:
                    print(f"{r}: {self.resources[r]}")
                elif isinstance(self.resources[r],int) and self.resources[r] > 0:
                    print(f"{r}: {self.resources[r]}")
            print("")
            print(f"{self.name}'s special resources:")
            empty = True
            for r in self.special_resources:
                if not isinstance(self.special_resources[r],int) and len(self.special_resources[r]) != 0:
                    print(f"{r}: {self.special_resources[r]}")
                    empty = False
                elif isinstance(self.special_resources[r],int) and self.special_resources[r] > 0:
                    print(f"{r}: {self.special_resources[r]}")
                    empty = False
            if empty:
                print("none")
            
            if len(self.build) > 2:
                print("\nBuild chains: ")
                for i in range(2,len(self.build)):
                    build = list(self.build.keys())[i]
                    print(build + " -> ", end="")
                    print(', '.join(self.build[build]))
                
        if special:
            return self.special_resources        
        return self.resources
    
    def get_hand(self):
        return self.hand
    
    def is_ai(self):
        return self.is_ai
    
    def give_trade(self):
        if self.is_trading_left:
            self.left_neighbor.recieve_trade(self.left_coins,self.name)
        if self.is_trading_right:
            self.right_neighbor.recieve_trade(self.right_coins,self.name)
        self.resources["coin"] -= (self.left_coins + self.right_coins)
        self.is_trading_left = False
        self.is_trading_right = False
        
    def recieve_trade(self, coins, name):
        self.resources["coin"] += coins
        print(f"{self.name} recieved {coins} coins from {name}")
        
    def add_resources(self, resources, is_card=True, is_token=False):
        
        products = resources
        
        if is_card and not is_token:
            products = resources.get_product()
            if resources.get_build()[0] != "none":
                self.build[resources.get_name()] = resources.get_build()
                  
        
        for r in products:
            if r in self.resources:
                self.resources[r]+=1
            elif r in self.special_resources:
                self.special_resources[r]+=1
            elif is_token:
                if r in self.special_resources["tokens"]:
                    self.special_resources["tokens"][r] += 1
                else:
                    self.special_resources["tokens"][r] = 1                    
            elif "/" in r:
                split = r.split('/')
                choice = {split[0]:split[1]}
                print(f"choice______________ {self.resources["choice"]}")
                if self.resources["choice"]:
                    self.resources["choice"].update(choice)
                else:
                    self.resources["choice"]= choice
            else:
                self.special_resources[r] = "Built"
        print(f"resources_______________ {self.resources}")
    def view_hand(self):
        view = True
        print(f"{self.name}'s hand: ")
        for name in self.hand:
            card = self.hand[name]
            available = "Not available"
            if name in self.available_cards:
                available = "Available"
            elif name in self.trade_cards:
                available = "Available through trade"
            print(f"{card.get_name(view)}:\n\tCost: {card.get_cost(view)} ({available})\n\tProduces: {card.get_product(view)}\n\tChain: {card.get_build(view)}")
        
    def available(self,hand):
        rare_resource = ["glass","silk","paper"]
        for card in hand:
            check = self.resources.copy()
            available = True
            trade = {}
            card_cost = hand[card].get_cost().copy()
            
            if card not in self.build and "free" not in card_cost: #build chain
                for cost in card_cost:
                    if check[cost] > 0:
                        check[cost] -= 1
                        card_cost.remove(cost)
                    elif cost in rare_resource and check["any_rare"] > 0:
                        check["any_rare"] -=1
                        card_cost.remove(cost)
                    elif check["any"] > 0:
                        check["any"] -=1
                        card_cost.remove(cost)
                    elif cost in check["choice"]:
                        card_cost.remove(cost)
                        del check["choice"][cost]
                    elif cost in check["choice"].values():
                        position = list(check["choice"].values()).index(cost)
                        key_list = list(check["choice"].keys())
                        key = key_list[position]
                        card_cost.remove(cost)
                        del check["choice"][key]
                    else:
                        available = False 
            
            if available:
                self.available_cards[card] = hand[card]
            else:
                can_trade, trade = self.trade_available(card_cost)
                if can_trade:
                    self.trade_cards[card] = trade
                else:
                    self.unavailable[card] = hand[card]
        
        if not self.stage["available"]:
            self.stage_available()
        
        return self.available_cards, self.trade_cards, self.unavailable
    
    def trade_available(self, check):
        left_check = self.left_neighbor.get_resources().copy()
        right_check = self.right_neighbor.get_resources().copy()
        if len(left_check) == 0 or not isinstance(left_check, dict):
            print("ERROR")
            return
        if len(right_check) == 0 or not isinstance(right_check, dict):
            print("ERROR")
            return
        rare_resource = ["glass","silk","paper"]
        left_trade = {"market": 2,"post":2, "wood":0,"stone":0,"brick":0,"ore":0,"glass":0,"silk":0,"paper":0}
        right_trade = {"market": 2,"post":2, "wood":0,"stone":0,"brick":0,"ore":0,"glass":0,"silk":0,"paper":0}
        left_cost = 0
        right_cost = 0
        if "left_trade" in self.special_resources:
            left_trade["post"] = 1
        if "right_trade" in self.special_resources:
            right_trade["post"] = 1
        if "market" in self.special_resources:
            left_trade["market"] = 1
            right_trade["market"] = 1
        
        
        for cost in check:
            left_has_trade = False
            right_has_trade = False
            
            #can only trade for basic or rare resources and no coin
            if cost not in left_trade: 
                return False, {}
            
            if left_check[cost] > 0:
                left_check[cost] -= 1
                left_trade[cost] += 1
                left_has_trade = True
            
            elif left_check["choice"]:
                if cost in left_check["choice"]:
                    left_cost += left_trade["post"]
                    left_has_trade = True
                    del left_check["choice"]
                elif cost in left_check["choice"].values():
                    position = list(left_check["choice"].values()).index(cost)
                    key_list = list(left_check["choice"].keys())
                    key = key_list[position]
                    left_has_trade = True
                    del left_check["choice"][key]
                
            if left_has_trade:
                if cost in rare_resource:
                    left_cost += left_trade["market"]
                else:
                    left_cost += left_trade["post"]
            
            print(f"right check _____________________ {right_check}")
            if right_check[cost] > 0:
                right_check[cost] -= 1
                right_trade[cost] += 1
                right_has_trade = True
            
            elif right_check["choice"]:
                if cost in right_check["choice"]:
                    right_cost += right_trade["post"]
                    right_has_trade = True
                    del right_check["choice"]
                elif cost in right_check["choice"].values():
                    position = list(right_check["choice"].values()).index(cost)
                    key_list = list(right_check["choice"].keys())
                    key = key_list[position]
                    right_has_trade = True
                    del right_check["choice"][key]
            if right_has_trade:
                if cost in rare_resource:
                    right_cost += right_trade["market"]
                else:
                    right_cost += right_trade["post"]
            
            if not (left_has_trade or right_has_trade):
                return False, {}
            else:
                if not left_has_trade or left_cost > self.resources["coin"]:
                    left_trade = {}
                if not right_has_trade or right_cost > self.resources["coin"]:
                    right_trade = {}
                if not (right_trade or left_trade):
                    return False, {}
        return True, {'left':left_trade, 'right':right_trade}
    
    def stage_available(self):
        check = self.resources.copy()
        available = True
        trade = {}
        
        stage_cost = self.board.get_stage_cost(self.board.get_stage()) 
        for cost in stage_cost:
            if check[cost] > 0:
                check[cost] -= 1
                stage_cost.remove(cost)
            else:
                available = False 
                can_trade, trade = self.trade_available(stage_cost)
                break
        if available:
            self.stage["available"] = True
        elif can_trade:
            self.stage["trade"] = trade
    
    def view(self):
        human = "Human"
        if self.is_ai:
            human = "AI"
        print(f"({human})")
        print(f"Name: {self.get_name()}")
        print(f"Board: {self.board.get_name()}")

In [320]:
class SevenWonders:
    def __init__(self, num_players=3,num_human=1,expansions=["base"]):
        self.num_players=num_players
        self.num_human=num_human
        self.expansions=expansions
        self.rand_boards=True
        self.Players = {}
        self.decks = []
        
    def setup(self):
        player_names = []
        
        if self.num_human > 0:
            for human in range(0,self.num_human):
                print(f"Enter name for player {human+1}:")
                player_names.append(input())

                
        confirm = False
        while not confirm:
            choice = input("Play with random boards? y/n ")
            if choice.lower() == "y" or choice.lower() == "yes" or choice == "0":
                self.rand_boards = True
                confirm = True
            elif choice.lower() == "n" or choice.lower() == "no" or choice == "1":
                self.rand_boards = False
                confirm = True
            else:
                print("please answer with yes or no")
                    
        for ai in range(0,(self.num_players - self.num_human)):            
            name = "ai" + str(ai+1)
            player_names.append(name)

        self.set_decks(self.expansions)
        self.set_boards(player_names, self.expansions)
        
        print("List of Players:")
        for player in self.Players:
            self.Players[player].view()
            print()
        self.set_neighbors(player_names)
        title = open("title_card.txt","r")
        print(title.read())

    def set_decks(self, expansions="base"):
        card_list = pd.read_csv("card _list.csv", skip_blank_lines=False)
        age1_list = card_list.loc[(card_list['age']==1) & (card_list['player_count'] <= self.num_players)]
        age2_list = card_list.loc[(card_list['age']==2) & (card_list['player_count'] <= self.num_players)]
        age3_list = card_list.loc[(card_list['age']==3) & (card_list['player_count'] <= self.num_players) & (card_list['color'] != "purple")]
        purple = card_list.loc[card_list['color'] == "purple"]
        age3_list = pd.concat([purple.sample(n=(self.num_players+2)),age3_list])
        age_lists = [age1_list,age2_list,age3_list]
        age1 = []
        age2 = []
        age3 = []
        self.decks = [age1, age2,age3]
        for age in range(0,3):
            for index,card in age_lists[age].iterrows():
                self.decks[age].append(Card(card["age"],card["name"],card["cost"],card["build"],card["product"],card["color"]))
        #if("Cities" in expansion):
        #    cities_cards = pd.read_csv("cities_card_list")
        #    (rest of code)
        return self.decks
        
    def set_boards(self, player_names, expansions="base"):
        board_list = pd.read_csv('board_list.csv', skip_blank_lines=False)
        board_frame = ""
        for human in range(0,self.num_human):
            name = player_names[human]
            if(self.rand_boards):
                board_frame = board_list.sample(replace=False)
                board_list.drop(board_frame.index, inplace=True)
            else:
                board_name = ""
                while board_name not in board_list["name"].values:
                    print(f"{name}, please select a board from the list below:")
                    print(board_list["name"].to_string(index=False))
                    board_name = input()
                    if board_name not in board_list["name"].values:
                        print(f"{board_name} isn't a board")
                board_frame = board_list.loc[(board_list["name"] == board_name)]
                board_list.drop(board_list[board_list["name"] == board_name].index, inplace=True)
            
            board = Board(board_frame["name"],board_frame["start"],board_frame["num_stages"],board_frame["costs"],board_frame["rewards"],board_frame["time"])
            self.Players[name] = Player(name, board, ai=False)
        
        num_ai = self.num_players-self.num_human
        for ai in range(self.num_players-num_ai, self.num_players):
            board_frame = board_list.sample(replace=False)
            board_list.drop(board_frame.index, inplace=True)
            board = Board(board_frame["name"],board_frame["start"],board_frame["num_stages"],board_frame["costs"],board_frame["rewards"],board_frame["time"])
            name = player_names[ai]
            self.Players[name] = Player(name, board, ai=True)
    
    def set_neighbors(self, player_names):
        n_edges = 0
        for player in range(0,self.num_players - 1):
            name = player_names[player]
            left = ""
            right = ""
            left_set = False
            right_set = False
            for n in player_names:
                print()
            while not left_set and n_edges < self.num_players:
                if self.Players[name].is_ai:
                    left = player_names[player + 1]
                    left_set = True
                else:
                    print(f"{name} who is sitting on your left?")
                    left = input()
                    if left not in player_names:
                        print(f"{left} is not a player")
                    elif left == name:
                        print("You can't pick yourself")
                    elif self.Players[left].get_neighbor("right") and n_edges < self.num_players - 1:
                        print(f"inner cycle detected. choose another player")
                    elif self.Players[left].get_neighbor("right"):
                        print(f"{left} already has a neighbor to their right. choose another player")
                    else:
                        left_set = True
            
            if n_edges < self.num_players:
                self.Players[name].set_left(self.Players[left])
                self.Players[left].set_right(self.Players[name])
            n_edges += 1
            
            while not right_set and n_edges < self.num_players:
                
                if self.Players[name].is_ai:
                    print(self.Players[name])
                    right = player_names[player - 1]
                    if player == 0:
                        right = player_names[len(player_names) - player]
                    right_set = True
                else:
                    print(f"{name} who is sitting on your right?")
                    right = input()
                    if right not in player_names:
                        print(f"{right} is not a player")
                    elif right == name:
                        print("You can't pick yourself")
                    elif self.Players[right].get_neighbor("left") and n_edges < self.num_players:
                        print(f"inner cycle detected. choose another player")
                    elif self.Players[right].get_neighbor("left"):
                        print(f"{right} already has a neighbor to their right. choose another player")
                    else:
                        right_set = True
            if n_edges < self.num_players:
                self.Players[name].set_right(self.Players[right])
                self.Players[right].set_left(self.Players[name])
            n_edges += 1
    
    def resolve_conflicts(self, age):
        tokens = ["age 1","age 2", "age 3", "deafeat"]
        winners = {}
        tie = {}
        for name in self.Players:
            player = self.Players[name]
            mp = player.get_resources(special=True)["military"]
            left = player.get_neighbor("left")
            right = player.get_neighbor("right")
            left_mp = left.get_resources(special=True)["military"]
            right_mp = right.get_resources(special=True)["military"]
            
            
            if mp > left_mp:
                winners[name] = {"left":left.get_name()}
                player.add_resources([tokens[age]], is_token=True)

            elif mp < left_mp:
                player.add_resources([tokens[3]], is_token=True) #adds defeat
            else:
                tie[name] = {"left":left.get_name()}
                    
            if mp > right_mp:
                if name in winners:
                    winners[name].update({"right":right.get_name()})
                else:
                    winners[name] = {"right":right.get_name()}
                player.add_resources([tokens[age]], is_token=True)
            elif mp < right_mp:
                player.add_resources([tokens[3]], is_token=True)
            else:
                if name in tie:
                    tie[name].update({"right":right.get_name()})
                else:
                    tie[name] = {"right":right.get_name()}
                
        for name in self.Players:
            if name in winners:
                if len(winners[name]) > 1:
                    print(f"{name} won their conflict against {left.get_name()} and {right.get_name()}!")
                else:
                    print(f"{name} won their conflict against {list(list(winners[name].keys())[0].keys())[0]}")
            if name in tie:
                if len(tie[name]) > 1:
                    print(f"In their conflict against {left.get_name()} and {right.get_name()}, {name} tied with both!")
                else:
                    print(f"{name} was tied in their conflict against {list(list(tie[name].keys())[0].keys())[0]}")

    def play(self):
        if not self.Players:
            print("Please setup players first")
            return
        hand = {}
        first_hand = {}
        num_cards = len(self.decks[0]) #all ages should have the same # of cards
        initial_hand_size = int(num_cards/self.num_players)
        action_list = {1:"played", 2:"discarded", 3:"built a stage of their wonder with"}
        for age in range(0,3):
            #sets initial hand for each age
            for n in self.Players:
                hand = {}
                cards = random.sample(self.decks[age],k=initial_hand_size)
                for c in cards:
                    self.decks[age].remove(c)
                for card in cards: #formats hand for each card be searched by card name
                    hand[card.get_name()] = card
                self.Players[n].set_hand(hand)
                
            turn = 0
            while turn < initial_hand_size - 1:
                action = 0
                card = ""
                old_hands = {}
                turn_actions = {}
                turn_cards = {}
                for name in self.Players:
                    player = self.Players[name]
                    action, card = player.set_action()
                    print(f"{player.get_name()} {action_list[action]} {card}")
                    turn_actions[name] = action
                    turn_cards[name] = card
                    
                for name in self.Players:
                    player = self.Players[name]
                    old_hands[name] = player.play_card(turn_actions[name], turn_cards[name])
                    player.give_trade()

                    if (age == 0 or age == 2):
                        right = player.get_neighbor("right")
                        if right.get_name() in old_hands:
                            new_hand = old_hands[right.get_name()]
                        else:
                            new_hand = right.get_hand()
                    else:
                        left = player.get_neighbor("left")
                        if left.get_name() in old_hands:
                            new_hand = old_hands[left.get_name()]
                        else:
                            new_hand = right.get_hand()
                        
                    self.Players[name].set_hand(new_hand)
                turn += 1
            self.resolve_conflicts(age)
                                    

In [ ]:
sw = SevenWonders()
sw.setup()
sw.play()

sw = SevenWonders()
decks = sw.set_decks()
